Name: Cornelius Adejoro
Lab Assignmet2

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [2]:
from tensorflow import keras
from keras.utils import to_categorical
from keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os
import pandas as pd
import time
from sklearn.metrics import precision_score, recall_score
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras import regularizers

Part2

In [3]:
!kaggle datasets download -d puneet6060/intel-image-classification

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile

# Define the path to the uploaded zip file
zip_file_path2 = "/content/intel-image-classification.zip"

# Extract the zip file
with zipfile.ZipFile(zip_file_path2, 'r') as zip_ref:
    zip_ref.extractall("/content/intel_dataset")

In [5]:
import pathlib

# Define directories
train_dir = "/content/intel_dataset/seg_train/seg_train"
test_dir = "/content/intel_dataset/seg_test/seg_test"

# Load the dataset and split into train, validation, and test sets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(32, 32),  # Set the desired image size
    batch_size=32,
    shuffle=True,
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(32, 32),
    batch_size=32,
    shuffle=False,  # No need to shuffle the test set
)

# Get class names (labels)
class_names = train_dataset.class_names

# Extract X_train, y_train, X_val, y_val
X_train = []
y_train = []
for images, labels in train_dataset:
    X_train.append(images.numpy())
    y_train.append(labels.numpy())

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)


# For the test set, you can directly use the test_dataset
X_test = []
y_test = []
for images, labels in test_dataset:
    X_test.append(images.numpy())
    y_test.append(labels.numpy())

X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)


Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [6]:
# Convert labels to one-hot encoded categorical format
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)

In [8]:
#Baseline model

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train = X_train.reshape(X_train.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data


# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
start = time.time()
model_base_01 = model.fit(X_train, y_train, epochs=150, batch_size=32)


# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.1936 - loss: 9.0694
Epoch 2/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1731 - loss: 1.7957
Epoch 3/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1864 - loss: 1.7950
Epoch 4/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1796 - loss: 1.8269
Epoch 5/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1800 - loss: 1.7913
Epoch 6/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1873 - loss: 1.7899
Epoch 7/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1812 - loss: 1.7907
Epoch 8/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1834 - loss: 1.7902
Epoch 9/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1839 - loss: 1.7903
Epoch 10/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1808 - loss: 1.7906
Epoch 11/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1755 - loss: 1.7907
Epoch 12/150
439/439 ━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Measuring baseline performance using 25%, 50% and 75% train data

In [7]:
# Select only 25% of X_train, y_train
X_train_25 = X_train[:int(0.5 * len(X_train))]
y_train_25 = y_train[:int(0.5 * len(y_train))]

# Select only 50% of X_train, y_train
X_train_50 = X_train[:int(0.25 * len(X_train))]
y_train_50 = y_train[:int(0.25 * len(y_train))]

# Select only 75% of X_train, y_train
X_train_75 = X_train[:int(0.75 * len(X_train))]
y_train_75 = y_train[:int(0.75 * len(y_train))]


In [10]:
#Baseline 25% train data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_25 = X_train_25.reshape(X_train_25.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data


# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
start = time.time()
model_base_02 = model.fit(X_train_25, y_train_25, epochs=150, batch_size=32)



# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1817 - loss: 12.9121
Epoch 2/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1734 - loss: 1.7970
Epoch 3/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1771 - loss: 1.8118
Epoch 4/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2232 - loss: 1.7751
Epoch 5/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2806 - loss: 1.6805
Epoch 6/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2880 - loss: 1.6488
Epoch 7/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2756 - loss: 1.6531
Epoch 8/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2923 - loss: 1.6123
Epoch 9/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2643 - loss: 1.6668
Epoch 10/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2916 - loss: 1.6110
Epoch 11/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2871 - loss: 1.5786
Epoch 12/150
220/220 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
#Baseline 50% train data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_50 = X_train_50.reshape(X_train_50.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data


# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
start = time.time()
model_base_03 = model.fit(X_train_50, y_train_50, epochs=150, batch_size=32)


# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.1878 - loss: 38.6844
Epoch 2/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1973 - loss: 2.1589
Epoch 3/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2141 - loss: 1.8081
Epoch 4/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2668 - loss: 1.7123
Epoch 5/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2928 - loss: 1.6592
Epoch 6/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2777 - loss: 1.6063
Epoch 7/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3167 - loss: 1.5499
Epoch 8/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2959 - loss: 1.5412
Epoch 9/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3291 - loss: 1.5494
Epoch 10/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3319 - loss: 1.5235
Epoch 11/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3253 - loss: 1.5116
Epoch 12/150
110/110 ━━━━━━━━━━━━━━━━━━

In [12]:
#Baseline 75% train data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_75 = X_train_75.reshape(X_train_75.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
start = time.time()
model_base_04 = model.fit(X_train_75, y_train_75, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1911 - loss: 12.9371
Epoch 2/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3469 - loss: 1.5787
Epoch 3/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4423 - loss: 1.4288
Epoch 4/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4418 - loss: 1.4120
Epoch 5/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4562 - loss: 1.3562
Epoch 6/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4740 - loss: 1.3522
Epoch 7/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4896 - loss: 1.3099
Epoch 8/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4853 - loss: 1.3176
Epoch 9/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4760 - loss: 1.3225
Epoch 10/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4869 - loss: 1.3151
Epoch 11/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5087 - loss: 1.2723
Epoch 12/150
329/329 ━━━━━━━━━━━━━━━━━━━

## Experiment 1: Architectural Regularization

In [9]:
#Baseline 25% train data with dropout

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_25 = X_train_25.reshape(X_train_25.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
start = time.time()
model_base_05 = model.fit(X_train_25, y_train_25, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.1700 - loss: 15.2531
Epoch 2/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1761 - loss: 1.8364
Epoch 3/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1787 - loss: 1.8305
Epoch 4/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1764 - loss: 1.7918
Epoch 5/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1831 - loss: 1.7905
Epoch 6/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1839 - loss: 1.7904
Epoch 7/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1820 - loss: 1.7904
Epoch 8/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1785 - loss: 1.7901
Epoch 9/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1899 - loss: 1.7900
Epoch 10/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1851 - loss: 1.7908
Epoch 11/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1754 - loss: 1.7900
Epoch 12/150
220/220 ━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
#Baseline 50% train data with dropout

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_50 = X_train_50.reshape(X_train_50.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
start = time.time()
model_base_06 = model.fit(X_train_50, y_train_50, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.1524 - loss: 30.7052
Epoch 2/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2028 - loss: 1.7887
Epoch 3/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1866 - loss: 1.7865
Epoch 4/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1798 - loss: 1.7887
Epoch 5/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1780 - loss: 1.7905
Epoch 6/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1878 - loss: 1.7897
Epoch 7/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1756 - loss: 1.7901
Epoch 8/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1791 - loss: 1.7918
Epoch 9/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1721 - loss: 1.7900
Epoch 10/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1864 - loss: 1.7879
Epoch 11/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1822 - loss: 1.7899
Epoch 12/150
110/110 ━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
#Baseline 75% train data with dropout

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_75 = X_train_75.reshape(X_train_75.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data


# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
start = time.time()
model_base_07 = model.fit(X_train_75, y_train_75, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.1697 - loss: 16.7581
Epoch 2/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1778 - loss: 1.7923
Epoch 3/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1728 - loss: 1.7927
Epoch 4/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1709 - loss: 1.7919
Epoch 5/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1745 - loss: 1.7916
Epoch 6/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1799 - loss: 1.7913
Epoch 7/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1744 - loss: 1.7912
Epoch 8/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1729 - loss: 1.7919
Epoch 9/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1890 - loss: 1.7902
Epoch 10/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1774 - loss: 1.7907
Epoch 11/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1832 - loss: 1.7910
Epoch 12/150
329/329 ━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
#Baseline 100% train data with dropout

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train = X_train.reshape(X_train.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
start = time.time()
model_base_08 = model.fit(X_train, y_train, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_32 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_28 (Dropout)                 │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_29 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_30 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_32 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_33 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_34 (Dropout)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.1616 - loss: 13.6768
Epoch 2/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1676 - loss: 1.7949
Epoch 3/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1830 - loss: 1.7911
Epoch 4/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1740 - loss: 1.7919
Epoch 5/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1763 - loss: 1.7917
Epoch 6/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1815 - loss: 1.7911
Epoch 7/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1711 - loss: 1.7914
Epoch 8/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1782 - loss: 1.7909
Epoch 9/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1741 - loss: 1.7915
Epoch 10/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1818 - loss: 1.7906
Epoch 11/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1774 - loss: 1.7912
Epoch 12/150
439/439 ━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Experiment 2: Data/Loss Regulization

In [13]:
#Adding Addergrad on 25% data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_25 = X_train_25.reshape(X_train_25.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", optimizer="Adagrad",  metrics=["accuracy"])
start = time.time()
model_base_09 = model.fit(X_train_25, y_train_25, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_40 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2581 - loss: 3.6607
Epoch 2/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3943 - loss: 1.5100
Epoch 3/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4504 - loss: 1.3947
Epoch 4/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4777 - loss: 1.3326
Epoch 5/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4848 - loss: 1.2905
Epoch 6/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5125 - loss: 1.2601
Epoch 7/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5180 - loss: 1.2384
Epoch 8/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5216 - loss: 1.2125
Epoch 9/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5329 - loss: 1.1907
Epoch 10/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5412 - loss: 1.1602
Epoch 11/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5401 - loss: 1.1628
Epoch 12/150
220/220 ━━━━━━━━━━━━━━━━━━━━

In [14]:
#Adding Addergrad on 50% data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_50 = X_train_50.reshape(X_train_50.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", optimizer="Adagrad",  metrics=["accuracy"])
start = time.time()
model_base_10 = model.fit(X_train_50, y_train_50, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_49 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_50 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_51 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_52 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_53 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_54 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_55 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.2053 - loss: 4.9223
Epoch 2/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3081 - loss: 1.6269
Epoch 3/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3808 - loss: 1.5143
Epoch 4/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3921 - loss: 1.4620
Epoch 5/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4310 - loss: 1.4190
Epoch 6/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4535 - loss: 1.3890
Epoch 7/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4559 - loss: 1.3561
Epoch 8/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4741 - loss: 1.3269
Epoch 9/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4914 - loss: 1.2886
Epoch 10/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4897 - loss: 1.2696
Epoch 11/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5209 - loss: 1.2417
Epoch 12/150
110/110 ━━━━━━━━━━━━━━━━━━━

In [ ]:
#Adding Addergrad on 75% data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_75 = X_train_75.reshape(X_train_75.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", optimizer="Adagrad",  metrics=["accuracy"])
start = time.time()
model_base_11 = model.fit(X_train_75, y_train_75, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_56 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_57 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_58 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_59 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_60 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_61 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_62 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_63 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2889 - loss: 2.4276
Epoch 2/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4329 - loss: 1.4289
Epoch 3/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4632 - loss: 1.3712
Epoch 4/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4909 - loss: 1.3080
Epoch 5/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5055 - loss: 1.2587
Epoch 6/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5169 - loss: 1.2313
Epoch 7/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5170 - loss: 1.2056
Epoch 8/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5293 - loss: 1.1928
Epoch 9/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5512 - loss: 1.1627
Epoch 10/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5447 - loss: 1.1510
Epoch 11/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5633 - loss: 1.1308
Epoch 12/150
329/329 ━━━━━━━━━━━━━━━━━━━━

In [16]:
#Adding Addergrad on 100% data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation="softmax"))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train = X_train.reshape(X_train.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data


# Train the model (configure + train)
model.compile(loss="categorical_crossentropy", optimizer="Adagrad",  metrics=["accuracy"])
start = time.time()
model_base_12 = model.fit(X_train, y_train, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_64 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_65 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_66 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_67 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_68 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_69 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_70 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_71 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2953 - loss: 2.5158
Epoch 2/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4293 - loss: 1.4279
Epoch 3/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4787 - loss: 1.3213
Epoch 4/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4983 - loss: 1.2833
Epoch 5/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5194 - loss: 1.2315
Epoch 6/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5309 - loss: 1.2079
Epoch 7/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5328 - loss: 1.1837
Epoch 8/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5478 - loss: 1.1639
Epoch 9/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5590 - loss: 1.1421
Epoch 10/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5636 - loss: 1.1181
Epoch 11/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5708 - loss: 1.1011
Epoch 12/150
439/439 ━━━━━━━━━━━━━━━━━━━━

## Experiment 2: Using l2 weight penalityz

In [17]:
#Adding regularizer on 25% data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(6, activation="softmax", kernel_regularizer=regularizers.l2(0.01)))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_25 = X_train_25.reshape(X_train_25.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy",  metrics=["accuracy"])
start = time.time()
model_base_13 = model.fit(X_train_25, y_train_25, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_72 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_73 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_74 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_75 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_76 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_77 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_78 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_79 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.2030 - loss: 24.2085
Epoch 2/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3087 - loss: 2.5936
Epoch 3/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4029 - loss: 1.7398
Epoch 4/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4488 - loss: 1.5816
Epoch 5/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4516 - loss: 1.5320
Epoch 6/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4649 - loss: 1.4910
Epoch 7/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4864 - loss: 1.4449
Epoch 8/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4710 - loss: 1.4735
Epoch 9/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4812 - loss: 1.4624
Epoch 10/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4731 - loss: 1.4585
Epoch 11/150
220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4821 - loss: 1.4442
Epoch 12/150
220/220 ━━━━━━━━━━━━━━━━━━

In [8]:
#Adding L2 regularizer on 50% data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(6, activation="softmax", kernel_regularizer=regularizers.l2(0.01)))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_50 = X_train_50.reshape(X_train_50.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy",  metrics=["accuracy"])
start = time.time()
model_base_14 = model.fit(X_train_50, y_train_50, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.1678 - loss: 42.4074
Epoch 2/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2119 - loss: 6.8139
Epoch 3/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1698 - loss: 2.6002
Epoch 4/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2853 - loss: 1.8300
Epoch 5/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3051 - loss: 1.6896
Epoch 6/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2969 - loss: 1.6603
Epoch 7/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3044 - loss: 1.6497
Epoch 8/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3125 - loss: 1.6363
Epoch 9/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3226 - loss: 1.6353
Epoch 10/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3096 - loss: 1.6464
Epoch 11/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3148 - loss: 1.6030
Epoch 12/150
110/110 ━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
#Adding regularizer on 75% data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(6, activation="softmax", kernel_regularizer=regularizers.l2(0.01)))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train_75 = X_train_75.reshape(X_train_75.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy",  metrics=["accuracy"])
start = time.time()
model_base_15 = model.fit(X_train_75, y_train_75, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_88 (Dense)                     │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_89 (Dense)                     │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_90 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_91 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_92 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_93 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_94 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_95 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.2060 - loss: 19.0519
Epoch 2/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3648 - loss: 1.8166
Epoch 3/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4340 - loss: 1.5648
Epoch 4/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4542 - loss: 1.5123
Epoch 5/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4744 - loss: 1.4658
Epoch 6/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4728 - loss: 1.4506
Epoch 7/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4685 - loss: 1.4744
Epoch 8/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4620 - loss: 1.4563
Epoch 9/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4791 - loss: 1.4400
Epoch 10/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4825 - loss: 1.4438
Epoch 11/150
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4932 - loss: 1.4163
Epoch 12/150
329/329 ━━━━━━━━━━━━━━━━━━━

In [9]:
#Adding regularizer on 100% data

# Initialize the model
model = Sequential()

# Define layers
model.add(Dense(50, activation='relu',  input_shape=(32*32*3,), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(6, activation="softmax", kernel_regularizer=regularizers.l2(0.01)))

# Print summary
model.summary()

# Reshape X_train to match the expected input shape
X_train = X_train.reshape(X_train.shape[0], 32*32*3) # Flatten the input data
X_test = X_test.reshape(X_test.shape[0], 32*32*3) # Flatten the input data



# Train the model (configure + train)
model.compile(loss="categorical_crossentropy",  metrics=["accuracy"])
start = time.time()
model_base_16 = model.fit(X_train, y_train, epochs=150, batch_size=32)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted')
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted')

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 50)                  │         153,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 32)                  │           1,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 6)                   │           6,150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 861,784 (3.29 MB)

 Trainable params: 861,784 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.1832 - loss: 16.9597
Epoch 2/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1735 - loss: 1.7914
Epoch 3/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1804 - loss: 1.7908
Epoch 4/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1855 - loss: 1.7902
Epoch 5/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1774 - loss: 1.7911
Epoch 6/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1796 - loss: 1.7908
Epoch 7/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1819 - loss: 1.7902
Epoch 8/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1753 - loss: 1.7908
Epoch 9/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1732 - loss: 1.7912
Epoch 10/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1849 - loss: 1.7900
Epoch 11/150
439/439 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1758 - loss: 1.7910
Epoch 12/150
439/439 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D

# Load the VGG16 model without the top (final) layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model by adding your own dense layer for the desired output
x = Conv2D(32, kernel_size=3, activation='relu', padding="SAME", strides=(2,2))(base_model.output)
y = GlobalAveragePooling2D()(x)
output_layer = Dense(6, activation='softmax')(y)

# Combine the base model and the new dense layer to create the final model
FT_model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
FT_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the summary of the model
FT_model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# checkpoint
checkpoint = ModelCheckpoint("best_model", monitor="val_loss", save_best_only=True, mode="min")

In [ ]:
FT_history = FT_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data = (X_test, y_test),
                             callbacks=[checkpoint])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(FT_history.history["accuracy"], color = "blue", label = "Train Accuracy")
plt.plot(FT_history.history["val_accuracy"], color = "red", label = "Test Accuracy")
plt.title("Accuracy curve of a fine-tuned pre-trained model")
plt.ylabel("Accuracy")
plt.xlabel("Number of epochs")
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model

saved_model = load_model("best_model")

_, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
_, test_acc = saved_model.evaluate(X_test, y_test, verbose=0)

print("Train: {:.3f}, Test: {:.3f}".format(train_acc, test_acc))